In [15]:
!node --version

v14.18.0


In [7]:
import sys
sys.path.append("/Users/alexandrec/lux-ai-2021/Kaggle/simple")

import os
os.getcwd() 

'/Users/alexandrec/lux-ai-2021/Kaggle/simple'

In [8]:
!pip install kaggle-environments -U
!pip install torch
!pip install torchsummary
!pip install tensorboard

You should consider upgrading via the '/Users/alexandrec/jupyter-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/alexandrec/jupyter-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/alexandrec/jupyter-env/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/alexandrec/jupyter-env/bin/python -m pip install --upgrade pip' command.


In [9]:
from kaggle_environments import make
import numpy as np
import torch

In [10]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device: ", device)

Using device:  cpu


In [11]:
import torch
import copy
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchsummary import summary

class MapEncoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(10, 64, 8, 2)
        self.conv2 = torch.nn.Conv2d(64, 128, 4, 2)
        self.relu = torch.nn.ReLU()
        self.ffn_map = torch.nn.Linear(3200,512)
    def forward(self, x):
        y = self.conv1(x)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.relu(y)
        flat = y.view(-1, 3200)
        out = self.ffn_map(flat)
        return out

class LuxrNet(torch.nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.rnn = torch.nn.GRU(9, 128, 1, dropout=0, batch_first=True)
        self.mapEncoder = MapEncoder()
        self.relu = torch.nn.ReLU()
        self.joiner = torch.nn.Linear(640,512)
        self.output = torch.nn.Linear(512,output_dim)

    def forward(self, global_features, units_features, packed=False):
        bs = global_features.shape[0]
        h0 = torch.zeros(1, bs, 128).to(device)
        glob = self.mapEncoder(global_features)
        preds, hn = self.rnn(units_features, h0)
        if packed:
            preds, lens_unpacked = torch.nn.utils.rnn.pad_packed_sequence(preds, batch_first=True)
        seq_len = preds.shape[1]
        hidden_out = self.joiner(torch.cat((glob.unsqueeze(1).repeat(1,seq_len,1), preds), 2))
        out = self.output(self.relu(hidden_out))
        return [out[:,k,:] for k in range(seq_len)]
        # return out

class DDQN(torch.nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.online = LuxrNet(output_dim)
        self.target = copy.deepcopy(self.online)
        
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, state, mode, seq_lenghts=None):
        global_features = state[0]
        units_features = state[1]
        if seq_lenghts:
            units_features = torch.nn.utils.rnn.pack_padded_sequence(units_features, seq_lenghts, batch_first=True, enforce_sorted=False)   
            
        if mode == "online":
            return self.online(global_features, units_features, seq_lenghts)
        if mode == "target":
            return self.target(global_features, units_features, seq_lenghts)
   

t = torch.rand(32, 10, 32, 32).to(device)
t2 = torch.rand(32, 12, 9).to(device)
print(t2[:,0,:].dtype)

model = DDQN(8).to(device)
y = model([t,t2],"online")
print(y[0].shape)
modeltest = LuxrNet(6)



torch.float32
torch.Size([32, 8])


In [7]:
# optimizer.zero_grad()        
# loss, hidden = model(data, hidden, targets)
# loss.backward()

# torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
# optimizer.step()

In [12]:
# wrapper for observation preprocessing - features extraction from the game
import math, sys
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate

import torch
import numpy as np

# test agent 
from agent import agent

class EnvWrapper:
    def __init__(self, env, opponent):
        self.game_state = Game()
        self.env = env
        self.trainer = env.train([None,opponent])
        obs = self.trainer.reset()
        self.update_game_state(obs)


    def update_game_state(self, observation):
        """Update the local game state from observation"""

        if observation["step"] == 0:
            self.game_state = Game()
            self.game_state._initialize(observation["updates"])
            self.game_state._update(observation["updates"][2:])
            self.game_state.id = observation.player
        else:
            self.game_state._update(observation["updates"])
        self.obs = observation
        self.game_map = self.game_state.map
        self.player = self.game_state.players[observation.player]
        self.opponent = self.game_state.players[(observation.player + 1) % 2]
        self.width, self.height = self.game_map.width, self.game_map.height

        self.resource_tiles: list[Cell] = []
        self.city_tiles: list[Cell] = []
    
        for y in range(self.height):
            for x in range(self.width):
                cell = self.game_state.map.get_cell(x, y)
                if cell.has_resource():
                    self.resource_tiles.append(cell)
                if cell.citytile is not None:
                    self.city_tiles.append(cell.citytile)   

        return self.game_state
    
    def get_reward(self):
        """Workaround to get a correct reward"""
        player = self.player
        ct_count = sum([len(v.citytiles) for k, v in player.cities.items()])
        unit_count = len(self.game_state.players[player.team].units)
        return ct_count * 10000 + unit_count*10 # + self.obs["step"]*10
    

    def get_cell_resources(self, cell: Cell):
        wood = 0
        coal = 0
        uranium = 0
        if cell.has_resource():
            wood = cell.resource.amount if cell.resource.type == "wood" else 0
            coal = cell.resource.amount if cell.resource.type == 'coal' else 0
            uranium = cell.resource.amount if cell.resource.type == 'uranium' else 0
        return wood, coal, uranium
    
    def get_adjacent_resources(self, pos):
        wood = 0
        coal = 0
        uranium = 0
        x, y = (pos.x, pos.y)
        for i in range(-1,1):
            for j in range(-1,1):
                cell = self.game_map.get_cell(x+i,y+j)
                if cell.has_resource():
                    wood += cell.resource.amount if cell.resource.type == "wood" else 0
                    coal += cell.resource.amount if cell.resource.type == 'coal' else 0
                    uranium += cell.resource.amount if cell.resource.type == 'uranium' else 0
        return wood, coal, uranium
    
    def get_units_state(self):
        units = []
        for unit in self.player.units:
            if unit.can_act():
                cell = self.game_map.get_cell_by_pos(unit.pos)
                worker_type = 0 if unit.is_worker() else 1
                cargo = unit.get_cargo_space_left()
                pos_x = unit.pos.x
                pos_y = unit.pos.y
                is_in_city = cell.citytile != None
                adjacent_wood, adjacent_coal, adjacent_uranium = self.get_adjacent_resources(unit.pos)
                research = self.player.research_points
                units.append([worker_type, cargo, adjacent_wood, adjacent_coal, adjacent_uranium, is_in_city, research, pos_x, pos_y])
        return torch.tensor(units,dtype=torch.float32).to(device)     
    
    def get_cell_type(self, cell):
        cell_code = 0
        all_units = [self.player.units, self.opponent.units]
        for unit in self.player.units:
            if unit.pos == cell.pos:
                cell_code = 1
                if unit.is_cart:
                    cell_code +=0.5
        for unit in self.opponent.units:
            if unit.pos == cell.pos:
                cell_code = 2
                if unit.is_cart:
                    cell_code +=0.5
        return cell_code


    def get_global_state(self):
        state = torch.zeros(10,32,32, dtype=torch.float32).to(device)
        for x in range(self.width):
            for y in range(self.height):
                cell = self.game_map.get_cell(x,y)
                w, c, u = self.get_cell_resources(cell)
                cell_occupant = self.get_cell_type(cell)
                cityid = int(cell.citytile.cityid[2:])+1 if cell.citytile else 0
                city = (cell.citytile.team+1) if cell.citytile else 0
                fuel = (self.opponent.cities[cell.citytile.cityid].fuel if city == self.opponent.team+1
                       else self.player.cities[cell.citytile.cityid].fuel if city == self.player.team+1 else 0)
                road = cell.road
                state[:,x,y]= torch.tensor([w, c, u, cell_occupant, cityid, fuel, city, road, x, y]).to(device)
        return state


    def step(self, action):
        obs, _, done, info = self.trainer.step(action)
        game_state = self.update_game_state(obs)
        reward = self.get_reward()
        global_state = self.get_global_state()
        units_state = self.get_units_state()
        city_state = torch.tensor([]).to(device)
        return [global_state,units_state,city_state], reward, done, info

    def reset(self):
        obs = self.trainer.reset()
        self.update_game_state(obs)
        global_state = self.get_global_state()
        units_state = self.get_units_state()
        city_state = torch.tensor([]).to(device)
        return [global_state,units_state,city_state]
    
    def get_game_objects(self):
        return [self.player, self.city_tiles]
    
#TEST
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
print(env.reset())
env = EnvWrapper(env, agent)
state = env.reset()

game_objects = env.get_game_objects()

[{'action': [], 'reward': 0, 'info': {}, 'observation': {'remainingOverageTime': 60, 'step': 0, 'width': 32, 'height': 32, 'reward': 0, 'globalUnitIDCount': 2, 'globalCityIDCount': 2, 'player': 0, 'updates': ['0', '32 32', 'rp 0 0', 'rp 1 0', 'r wood 0 29 316', 'r wood 1 24 342', 'r wood 1 25 323', 'r uranium 1 31 332', 'r wood 2 24 329', 'r wood 2 25 397', 'r wood 2 26 364', 'r uranium 2 31 346', 'r wood 3 28 800', 'r wood 4 10 318', 'r wood 4 11 398', 'r uranium 4 17 336', 'r wood 4 27 800', 'r wood 4 28 800', 'r wood 5 10 308', 'r wood 5 11 326', 'r uranium 5 17 335', 'r coal 7 31 355', 'r coal 8 0 369', 'r coal 8 30 404', 'r coal 8 31 422', 'r wood 11 23 317', 'r wood 11 24 330', 'r wood 11 25 314', 'r wood 12 23 349', 'r wood 12 24 390', 'r wood 13 6 337', 'r wood 13 7 325', 'r wood 13 24 374', 'r wood 14 6 369', 'r wood 14 7 387', 'r coal 14 16 393', 'r coal 15 14 379', 'r coal 15 15 411', 'r coal 15 16 403', 'r coal 16 14 379', 'r coal 16 15 411', 'r coal 16 16 403', 'r wood 17 

In [13]:
import random
import sys
from lux.game import Game
from functools import partial
from collections import deque
from lux.constants import Constants
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_sequence
from torch.utils.tensorboard import SummaryWriter

DIRECTIONS = Constants.DIRECTIONS
class Bot:
    def __init__(self, save_dir):
        
        self.use_cuda = torch.cuda.is_available()
        self.save_dir = save_dir
        self.action_space = 6
        self.model = DDQN(self.action_space).to(device)
        self.memory = deque(maxlen=100000)
        
        self.exploration_rate = 1
        self.exploration_rate_decay = 0.9999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0
        self.save_every = 1e5
        self.batch_size = 16
        self.gamma = 0.9

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.SmoothL1Loss()
        
        self.burnin = 10000 #1e4  # min. experiences before training
        self.learn_every = 2  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync
        
        self.log_every = 30
        
        self.training = True

        # if self.use_cuda:
        #     self.net = self.net.to(device="cuda")

    def play(self, state, game_objects):
        """Play 1 step with epsilon-greedy policy and return actions"""
        actions = []
        cities_actions = []
        player, city_tiles = game_objects
        global_state_tensor = state[0].unsqueeze(0)
        units_state_tensor = state[1].unsqueeze(0)
        explore = np.random.rand() < self.exploration_rate and self.training

        def get_random_actions(unit):
            actions_list = [unit.move(DIRECTIONS.NORTH), unit.move(DIRECTIONS.SOUTH), unit.move(DIRECTIONS.EAST), 
                            unit.move(DIRECTIONS.WEST), unit.build_city(), unit.pillage()]
            i = np.random.randint(6, size=1)[0]
            return actions_list[i]

        num_citytiles = len([city for city in city_tiles if city.team == player.team])
        num_units = len(player.units)
        for city in city_tiles:
            # TODO: hardcoded city policy here
            if city.team == player.team and city.can_act():
                if num_units < num_citytiles:
                    num_units +=1
                    cities_actions.append(city.build_worker())
                else:
                    cities_actions.append(city.research())
                    

        # print(player.team)
        if (explore):
        #EXPLORE
            for unit in player.units:
                if unit.is_worker() and unit.can_act():
                    actions.append(get_random_actions(unit))
        else:
        # EXPLOIT
            if len(units_state_tensor) > 1:
                action_qvalues = self.model([global_state_tensor, units_state_tensor], mode="online")
                # print(action_qvalues)
                # select idx with higher Q prediction for each unit
                action_idx = [torch.argmax(Q, axis=1).item() for Q in action_qvalues]

                idx=0
                for unit in player.units:
                    if unit.can_act():
                        actions_list = [unit.move(DIRECTIONS.NORTH), unit.move(DIRECTIONS.SOUTH), unit.move(DIRECTIONS.EAST), 
                                unit.move(DIRECTIONS.WEST), unit.build_city(), unit.pillage()]
                        action = actions_list[action_idx[idx]]
                        actions.append(action)
                        idx+=1

        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1

        return actions, cities_actions

    def cache(self, state, next_state, action, reward, done, info):
        global_state = state[0]
        units_state = state[1]
        
        next_global_state = next_state[0]
        next_unit_state = next_state[1]
        
        action = torch.tensor(action).to(device)
        reward = torch.tensor([reward]).to(device)
        done = torch.tensor([done]).to(device)

        self.memory.append((global_state, units_state, next_global_state, next_unit_state, action, reward, done,))
    
    
    def get_batch(self):
        """Sample a batch from the cache"""
                       
        batch = random.sample(self.memory, self.batch_size)
        batch_constant = [(state[0], state[2], state[5], state[6]) for state in batch] # constant shape: global_state, next_global_state, reward, done
        
        batch_sequences = [(state[1], state[3], state[4]) for state in batch] # variable shape: units_state, next_units_state, action
        seq_lenghts = [(seq[0].shape[0], seq[1].shape[0], seq[2].shape[0]) for seq in batch_sequences]
        
        global_state, next_global_state, reward, done = map(torch.stack, zip(*batch_constant))
        unit_state, next_unit_state, action = map(lambda x: pad_sequence(x, batch_first=True), zip(*batch_sequences))
        
        return global_state, unit_state, next_global_state, next_unit_state, action, reward.squeeze(), done.squeeze(), seq_lenghts
    
    def save(self):
        save_path = (
            self.save_dir + f"LuxRN_net_{int(self.curr_step // self.save_every)}.chkpt"
        )
        torch.save(
            dict(model=self.model.state_dict(), exploration_rate=self.exploration_rate),
            save_path,
        )
        print(f"LuxRN saved to {save_path} at step {self.curr_step}")

    def learn(self):

        def td_estimate(state, actions, seq_lens):
            """Estimate for a sequence of units"""
            # action.register_hook(lambda grad : print(grad))
            Q_sequence = self.model(state, "online", seq_lens)
            # Q_sequence[0].register_hook(lambda grad : print("gradient:", grad))
            max_len = actions.shape[1]
            actions = torch.chunk(action, max_len, dim=1)
            current_Q_sequence = [torch.gather(current_Q, 1, action) for (action, current_Q) in zip(actions, Q_sequence)]
            return current_Q_sequence

        @torch.no_grad()
        def td_target(reward, next_state, done, seq_lens):
            """Target for a sequence of units"""
            next_state_Q_sequence = self.model(next_state, "online", seq_lens)
            # need to mask se sequence here?

            
            best_actions = [torch.argmax(next_state_Q, axis=1).unsqueeze(1) for next_state_Q in next_state_Q_sequence]
            next_Q_sequence = self.model(next_state, "target", seq_lens)
            next_best_Q_sequence = [torch.gather(current_Q, 1, action) for (action, current_Q) in zip(best_actions, next_Q_sequence)]

            return [(reward.unsqueeze(1) + (1 - done.unsqueeze(1).float()) * self.gamma * next_Q).float() for next_Q in next_best_Q_sequence]
        
        @torch.enable_grad()
        def update_Q_online(td_estimate, td_target):
            losses = torch.stack([self.loss_fn(td_e, td_t) for (td_e, td_t) in zip(td_estimate, td_target)])
            # losses.register_hook(lambda grad : print(grad))
            loss = torch.mean(losses)
            # loss.register_hook(lambda grad : print(grad))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            # print (self.model.online.rnn.weight_ih_l0.grad)
            return loss.item()

        def sync_Q_target():
            self.model.target.load_state_dict(self.model.online.state_dict())
            
            
        if self.curr_step % self.sync_every == 0:
            sync_Q_target()

        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None
        
        if self.curr_step == self.burnin:
            print("Learning started")
        

        if self.curr_step % self.learn_every != 0:
            return None, None
        
        # Sample from memory
        global_state, units_state, next_global_state, next_unit_state, action, reward, done, seq_lenghts = self.get_batch()
        state = [global_state, units_state]
        next_state = [next_global_state, next_unit_state]
        
        state_seq_lenghts = [sl[0] for sl in seq_lenghts]
        next_state_seq_lenghts = [sl[1] for sl in seq_lenghts]
        
        # Get TD Estimate
        td_est = td_estimate(state, action, state_seq_lenghts)

        # Get TD Target
        td_tgt = td_target(reward, next_state, done, next_state_seq_lenghts)
        # print(td_tgt[0].shape)
        # print(td_est[0].shape)

        # Backpropagate loss through Q_online
        loss = update_Q_online(td_est, td_tgt)
        

        # return (td_est.mean().item(), loss)
        return loss, None

    def simple_logging(self, to_log):
        # if self.curr_step % self.log_every == 0:
        for (k,v) in to_log.items():
            print(k, v)
            sys.stdout.flush()
    
    def log_tensorboard(self, writer):
        if self.curr_step > self.burnin:
            
            writer.add_histogram("online_GRU_0_ih", self.model.online.rnn.weight_ih_l0 , self.curr_step)
            writer.add_histogram("online_GRU_0_ih_bias", self.model.online.rnn.bias_ih_l0 , self.curr_step)
            writer.add_histogram("online_GRU_0_hh", self.model.online.rnn.weight_hh_l0 , self.curr_step)
            writer.add_histogram("online_GRU_0_hh_bias", self.model.online.rnn.bias_hh_l0 , self.curr_step)
            
            # writer.add_histogram("online_GRU_1_ih", self.model.online.rnn.weight_ih_l1 , self.curr_step)
            # writer.add_histogram("online_GRU_1_ih_bias", self.model.online.rnn.bias_ih_l1 , self.curr_step)
            # writer.add_histogram("online_GRU_1_hh", self.model.online.rnn.weight_hh_l1, self.curr_step)
            # writer.add_histogram("online_GRU_1_hh_bias", self.model.online.rnn.bias_hh_l1 , self.curr_step)
            
            writer.add_histogram("online_map_conv1", self.model.online.mapEncoder.conv1.weight, self.curr_step)
            writer.add_histogram("online_map_conv2", self.model.online.mapEncoder.conv2.weight, self.curr_step)
            writer.add_histogram("online_map_conv1_bias", self.model.online.mapEncoder.conv1.bias, self.curr_step)
            writer.add_histogram("online_map_conv2_bias", self.model.online.mapEncoder.conv2.bias, self.curr_step)
            
            writer.add_histogram("online_map_ffn", self.model.online.mapEncoder.ffn_map.weight, self.curr_step)
            writer.add_histogram("online_map_ffn_bias", self.model.online.mapEncoder.ffn_map.bias, self.curr_step)
            
            writer.add_histogram("online_joiner", self.model.online.joiner.weight, self.curr_step)
            writer.add_histogram("online_joiner_bias", self.model.online.joiner.bias, self.curr_step)
            
            writer.add_histogram("online_output", self.model.online.output.weight, self.curr_step)
            writer.add_histogram("online_output_bias", self.model.online.output.bias, self.curr_step)
            
            writer.add_scalar("Exploration_rate", self.exploration_rate, self.curr_step)
            
            #Grads
            
            writer.add_histogram("online_GRU_0_ih_grad", self.model.online.rnn.weight_ih_l0.grad , self.curr_step)
            writer.add_histogram("online_GRU_0_ih_bias_grad", self.model.online.rnn.bias_ih_l0.grad , self.curr_step)
            writer.add_histogram("online_GRU_0_hh_grad", self.model.online.rnn.weight_hh_l0.grad , self.curr_step)
            writer.add_histogram("online_GRU_0_hh_bias_grad", self.model.online.rnn.bias_hh_l0.grad , self.curr_step)
            
            writer.add_histogram("online_map_conv1_grad", self.model.online.mapEncoder.conv1.weight.grad, self.curr_step)
            writer.add_histogram("online_map_conv2_grad", self.model.online.mapEncoder.conv2.weight.grad, self.curr_step)
            writer.add_histogram("online_map_conv1_bias_grad", self.model.online.mapEncoder.conv1.bias.grad, self.curr_step)
            writer.add_histogram("online_map_conv2_bias_grad", self.model.online.mapEncoder.conv2.bias.grad, self.curr_step)
            
            writer.add_histogram("online_map_ffn_grad", self.model.online.mapEncoder.ffn_map.weight.grad, self.curr_step)
            writer.add_histogram("online_map_ffn_bias_grad", self.model.online.mapEncoder.ffn_map.bias.grad, self.curr_step)
            
            writer.add_histogram("online_joiner_grad", self.model.online.joiner.weight.grad, self.curr_step)
            writer.add_histogram("online_joiner_bias_grad", self.model.online.joiner.bias.grad, self.curr_step)
            
            writer.add_histogram("online_output_grad", self.model.online.output.weight.grad, self.curr_step)
            writer.add_histogram("online_output_bia_grads", self.model.online.output.bias.grad, self.curr_step)
            


bot = Bot( "/Users/alexandrec/lux-ai-2021/Kaggle/simple/ckpts")
bot.play(state, game_objects)
bot.save()




LuxRN saved to /Users/alexandrec/lux-ai-2021/Kaggle/simple/ckptsLuxRN_net_0.chkpt at step 1


In [20]:
from torch.utils.tensorboard import SummaryWriter
import time
torch.manual_seed(43)
np.random.seed(43)

episodes = 10
bot = Bot( "/Users/alexandrec/lux-ai-2021/Kaggle/simple/ckpts")
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": False}, debug=False)
env = EnvWrapper(env, agent)
writer = SummaryWriter()

def action_string_to_idx(action):
    offsets = {"n":0, "s":1, "e":2, "w":3, "b":4, "p":5}
    if action[0] == "m":
        return offsets[action[-1]]
    else:
        return offsets[action[0]]
    
max_reward = 1000000
best_episodes = []

for e in range (episodes):
    state = env.reset()
    game_objects = env.get_game_objects()
    episode_reward = 0
    t0 = time.time()
    while True:
        # Run agent on the state
        # Cities action are not learnable yet and must be separated
        action, cities_actions = bot.play(state, game_objects)
        # print(len(game_objects[0].units))
        # print(len(action))
        
        # Agent performs action
        next_state, reward, done, info = env.step([*action, *cities_actions])
        game_objects = env.get_game_objects()
        writer.add_scalar('reward', reward, bot.curr_step)

        
        # Remember if action is not empty and next state has units that can play
        if len(action)>0 and len(next_state[1])>0:
            action = [action_string_to_idx(a) for a in action]
            bot.cache(state, next_state, action, reward, done, info)

        # Learn
        loss, _ = bot.learn()
        # writer.add_scalar('loss', loss, env.obs["step"])
        
        episode_reward += reward

        # Checkpoint
        if bot.curr_step == 1 or bot.curr_step % bot.save_every == 0:
            bot.save()

        

        # Update state
        state = next_state
        if done:
            total_time = time.time() - t0
            bot.log_tensorboard(writer)
            writer.add_scalar('Episode_reward', episode_reward, bot.curr_step)
            writer.add_scalar('Episode_lengh', env.obs["step"], bot.curr_step)
            print ("Throughput: ", env.obs["step"]/ total_time)
            bot.simple_logging({"episode":e, "reward": episode_reward})
            # save best episodes for later
            if episode_reward > max_reward:
                max_reward = episode_reward
                best_episodes.append(env.env.toJSON())
                print(len(best_episodes))
            break
writer.close()

LuxRN saved to /Users/alexandrec/lux-ai-2021/Kaggle/simple/ckptsLuxRN_net_0.chkpt at step 1
Throughput:  31.9752246816511
episode 0
reward 2195430
1
Throughput:  31.130789360995443
episode 1
reward 600850
Throughput:  30.395128814362582
episode 2
reward 4105890
2
Throughput:  31.63554630341977
episode 3
reward 2563600
Throughput:  32.35350997198564
episode 4
reward 3675140
Throughput:  31.229948640793232
episode 5
reward 440430
Throughput:  31.678660324417127
episode 6
reward 1791910
Throughput:  32.098552693466125
episode 7
reward 661590
Throughput:  32.556444427072215
episode 8
reward 1001300
Throughput:  32.23302984993454
episode 9
reward 440730


In [ ]:
# import json
# print(len(best_episodes))
# for k, replay in enumerate(best_episodes):
#     with open(f"best_replay_{k}.json", "w") as f:
#         json.dump(replay, f)
# env.env.render(mode="ipython", width=1200, height=800)

In [56]:
# Emergency save
bot.save()

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [12]:

# env.render(mode="ipython", width=1200, height=800)

In [12]:
# test model predictions
bot.model.training = False
max_reward = 1000000
pred_best_episodes = []
for e in range (10):
    state = env.reset()
    game_objects = env.get_game_objects()
    episode_reward = 0
    while True:
        # Run agent on the state
        # Cities action are not learnable yet and must be separated
        action, cities_actions = bot.play(state, game_objects)
        # print(len(game_objects[0].units))
        # print(len(action))
        
        # Agent performs action
        next_state, reward, done, info = env.step([*action, *cities_actions])
        game_objects = env.get_game_objects()
        writer.add_scalar('reward', reward, bot.curr_step)

        # Learn
        loss, _ = bot.learn()
        # writer.add_scalar('loss', loss, env.obs["step"])
        episode_reward += reward

        # Update state
        state = next_state
        if done:        
            bot.simple_logging({"episode":e, "reward": episode_reward})
            # save best episodes for later
            if episode_reward > max_reward:
                max_reward = episode_reward
                pred_best_episodes.append(env.env.toJSON())
                print(len(best_episodes))
            break
writer.close()
bot.model.training = True

episode 0
reward 2274790
5
episode 1
reward 1423790
episode 2
reward 671160
episode 3
reward 1392010
episode 4
reward 2133550
episode 5
reward 550980
episode 6
reward 1191060
episode 7
reward 5236180
5
episode 8
reward 1321960
episode 9
reward 870820


In [21]:
import json
print(len(best_episodes))
for k, replay in enumerate(best_episodes):
    with open(f"best_replay_{k}.json", "w") as f:
        json.dump(replay, f)
env.env.render(mode="ipython", width=1200, height=800)

2


In [45]:
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print(x))  

b.mean().backward() 


print(a.grad, b.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [13]:
# TODO: 
# - reduce parameters


# - Recheck the loss to ignore the padded sequence
# - negative reward +-



